In [1]:
import os
import sys
from orgparse import load

import shutil
from pytube import YouTube


def makedir(d, usedir=True):
    if usedir:
        d = os.path.dirname(d)

    if not os.path.exists(d):
        os.makedirs(d)

def get_extension(stream):
    mime_type = stream.mime_type
    if 'mp4' in mime_type:
        return '.m4a'
    elif 'webm' in mime_type:
        return '.webm'        
    
def is_downloaded(output_prefix):
    possible_extensions = ['m4a', 'webm']
    for ext in possible_extensions:
        if os.path.exists(output_prefix + '.' + ext):
            return True
    return False    
# is_downloaded('./downloads/刘欢/弯弯的月亮'), is_downloaded('./downloads/刘欢/弯弯的月')    

In [36]:
YouTube('https://www.youtube.com/watch?v=GPnymcrXgX0').streams

[<Stream: itag="17" mime_type="video/3gpp" res="144p" fps="8fps" vcodec="mp4v.20.3" acodec="mp4a.40.2" progressive="True" type="video">, <Stream: itag="18" mime_type="video/mp4" res="360p" fps="30fps" vcodec="avc1.42001E" acodec="mp4a.40.2" progressive="True" type="video">, <Stream: itag="22" mime_type="video/mp4" res="720p" fps="30fps" vcodec="avc1.64001F" acodec="mp4a.40.2" progressive="True" type="video">, <Stream: itag="137" mime_type="video/mp4" res="1080p" fps="30fps" vcodec="avc1.640028" progressive="False" type="video">, <Stream: itag="248" mime_type="video/webm" res="1080p" fps="30fps" vcodec="vp9" progressive="False" type="video">, <Stream: itag="399" mime_type="video/mp4" res="1080p" fps="30fps" vcodec="av01.0.08M.08" progressive="False" type="video">, <Stream: itag="136" mime_type="video/mp4" res="720p" fps="30fps" vcodec="avc1.4d401f" progressive="False" type="video">, <Stream: itag="247" mime_type="video/webm" res="720p" fps="30fps" vcodec="vp9" progressive="False" type="

In [33]:
best_item = YouTube('https://www.youtube.com/watch?v=GPnymcrXgX0').streams.filter(
    only_audio=True, mime_type='audio/mp4'
).order_by('abr').desc().first()

KeyboardInterrupt: 

In [135]:

def recurse(node, root_dir='./downloads'):
    heading, body = node.heading, node.body
    if len(heading) > 0 and len(body) > 0:  # not an empty
        output_prefix = os.path.join(root_dir, heading)
        makedir(output_prefix, usedir=True)
        print('-' * 10)
        print(f'processing "{heading}"')        
        url = body.strip()
        
        if not is_downloaded(output_prefix):
            max_tries = 5  # TODO: be a argument
            succeeds = False
            for n_tries in range(1, max_tries+1):
                try:
                    streams = YouTube(url).streams.filter(only_audio=True)
                    succeeds = True
                    break
                except ConnectionResetError:
                    print(f'trying {n_tries} time(s)')
            
            if not succeeds:
                print(f'trying {max_tries} times still failed, skip it')
            else:
                stream = streams.first()
                output_path = output_prefix + get_extension(stream)
                print(f'downloading {url} to {output_path}')
                dirname, filename = os.path.dirname(output_path), os.path.basename(output_path)
                stream.download(dirname, filename=filename)
        else:
            print(f'{output_prefix} downloaded already')
    for child in node.children:   
        new_root_dir = (os.path.join(root_dir, heading) if len(heading) > 0 else root_dir)
        recurse(child, new_root_dir)
# shutil.rmtree('./downloads')



In [ ]:
x  